# OrthodoxAI - Agents

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros?tab=repositories<br>
> Year: 2025

## About the Project

This notebook is dedicated to prototyping and testing the behavior of individual and collaborative agents within the **OrthodoxAI** multi-agent system.

It serves as a sandbox to:

- Define agent roles (e.g., Retriever, Reflection, Generator)
- Simulate agent communication structure patters
- Experiment with routing logic and agent chaining (without LangGraph)
- Test prompt templates and output formatting for each agent
- Evaluate performance and consistency across multiple LLM-backed tasks

This notebook is used for isolated debugging and experimentation before integrating agents into the main production pipeline.

## Libraries

In [1]:
# Load the API Keys
import os
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv()
_ = load_dotenv(dotenv_path)

## OpenAI APIs

### Client Setup

* Import libraries to chat with OpenAI

In [2]:
from pydantic import BaseModel, Field
from typing import List, Literal

import openai
from openai import OpenAI

* Define a Structured Output template

In [3]:
class AnalyzerOutput(BaseModel):
    classification: Literal["Religious", "Non-Religious"] = Field(..., description="Either 'Religious' or 'Non-Religious'")
    key_topics: List[str] = Field(..., description="List of key topics/areas related to the user's question (e.g., theology, jesus, humility, virtues)",)
    context_requirements: str = Field(..., description="A clear explanation of the query's context needs")
    query_complexity: Literal["Low", "Medium", "High"] = Field(..., description="'Low', 'Medium', or 'High' complexity")
    reasoning: str = Field(..., description="The Chain of Thought that has been done in order to analyze the user query")

* Initialize configs

In [4]:
MODEL_NAME = 'o3-mini'

client = OpenAI()

chat_template = [
    {"role": "system", "content": "You are an AI assistant that classifies user queries as either religious or non-religious and extracts key topics."},
    {"role": "user", "content": "Helloo!!"}
]

### Simple response

In [5]:
response = client.chat.completions.create(
    model=MODEL_NAME, 
    messages=chat_template,
)

print(response.choices[0].message.content)

Classification: Non-religious

Key Topics:
• Greeting/Casual conversation


In [6]:
response = client.responses.create(
    model=MODEL_NAME,
    input=chat_template
)

print(response.output_text)

Classification: Non-religious

Key topics: Greeting, informal salutation

The message appears to be a friendly greeting without any religious context.


### Streaming response

In [7]:
response = ""

for chunk in client.chat.completions.create(
    model=MODEL_NAME,
    messages=chat_template,
    stream=True
):
    
    if not chunk.choices:
        continue  # Just a safeguard
    
    # The partial text is in 'delta.content'
    chunk_text = chunk.choices[0].delta.content
    if chunk_text is None:
        continue

    response += chunk_text
    print(chunk_text, end="", flush=True)

Classification: Non-religious

Key Topics: Greeting

The message appears to be a friendly, informal greeting with no religious content.

In [8]:
stream = client.responses.create(
    model=MODEL_NAME,
    input=chat_template,
    stream=True
)

for event in stream:
    if event.type == 'response.refusal.delta':
        print(event.delta, end="")
    elif event.type == 'response.output_text.delta':
        print(event.delta, end="")
    elif event.type == 'response.error':
        print(event.error, end="")
    elif event.type == 'response.completed':
        response = event.response.output

Classification: Non-religious

Key Topics: Greeting, informal salutation.

In [9]:
response

[ResponseReasoningItem(id='rs_67eefa37c2048191b2860f006b1d9261003da8076e6768b6', summary=[], type='reasoning', status=None),
 ResponseOutputMessage(id='msg_67eefa381b208191931e99bd52493d89003da8076e6768b6', content=[ResponseOutputText(annotations=[], text='Classification: Non-religious\n\nKey Topics: Greeting, informal salutation.', type='output_text')], role='assistant', status='completed', type='message')]

### Structured response

In [10]:
response = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=chat_template,
    response_format=AnalyzerOutput
)

structured_object = response.choices[0].message.parsed

In [11]:
structured_object

AnalyzerOutput(classification='Non-Religious', key_topics=['greeting'], context_requirements='No complexity is involved and there is no religious context present in the query. The query is simply a casual greeting.', query_complexity='Low', reasoning="The query 'Helloo!!' is a casual greeting with no mention of religious topics, symbols, or language. There is no indication of the conversation touching on religious themes, hence it is classified as non-religious and of low complexity.")

In [12]:
response.choices[0].message.content

'{\n  "classification": "Non-Religious",\n  "key_topics": [\n    "greeting"\n  ],\n  "context_requirements": "No complexity is involved and there is no religious context present in the query. The query is simply a casual greeting.",\n  "query_complexity": "Low",\n  "reasoning": "The query \'Helloo!!\' is a casual greeting with no mention of religious topics, symbols, or language. There is no indication of the conversation touching on religious themes, hence it is classified as non-religious and of low complexity."\n}'

In [13]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.tool_calls)
print(response.choices[0].message.role)
print(response.choices[0].message.refusal)

None
None
assistant
None


In [14]:
schema_dict = AnalyzerOutput.schema()
schema_dict["additionalProperties"] = False
text_dict = {
    "format": {
        "type": "json_schema",
        "name": "MyAnalyzerOutput",  # arbitrary name
        "strict": True,
        "schema": schema_dict
    }
}

response = client.responses.create(
    model=MODEL_NAME,
    input=chat_template,
    text=text_dict
)

/tmp/ipykernel_4511/789635851.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  schema_dict = AnalyzerOutput.schema()


In [15]:
response

Response(id='resp_67eefa3c1c608191a232cc4cc3cc9aa505b562a8227eead7', created_at=1743714876.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='o3-mini-2025-01-31', object='response', output=[ResponseReasoningItem(id='rs_67eefa3e69908191a96199dec3343ce505b562a8227eead7', summary=[], type='reasoning', status=None), ResponseOutputMessage(id='msg_67eefa3f67788191b6a07580c38dfaf905b562a8227eead7', content=[ResponseOutputText(annotations=[], text='{\n  "classification": "Non-Religious",\n  "key_topics": [\n    "greeting"\n  ],\n  "context_requirements": "The query is simply a casual greeting without any religious content or additional context.",\n  "query_complexity": "Low",\n  "reasoning": "The user message \'Helloo!!\' is a greeting and does not contain any content related to religious topics. Therefore, it is classified as non-religious with the key topic being a greeting."\n}', type='output_text')], role='assistant', status='completed', type='message')], parall

In [16]:
response.output_text

'{\n  "classification": "Non-Religious",\n  "key_topics": [\n    "greeting"\n  ],\n  "context_requirements": "The query is simply a casual greeting without any religious content or additional context.",\n  "query_complexity": "Low",\n  "reasoning": "The user message \'Helloo!!\' is a greeting and does not contain any content related to religious topics. Therefore, it is classified as non-religious with the key topic being a greeting."\n}'

In [17]:
response.output[1].content[0]

ResponseOutputText(annotations=[], text='{\n  "classification": "Non-Religious",\n  "key_topics": [\n    "greeting"\n  ],\n  "context_requirements": "The query is simply a casual greeting without any religious content or additional context.",\n  "query_complexity": "Low",\n  "reasoning": "The user message \'Helloo!!\' is a greeting and does not contain any content related to religious topics. Therefore, it is classified as non-religious with the key topic being a greeting."\n}', type='output_text')

### Multi-Step Structured Response

* Initialize the main template and the step's template

In [18]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [19]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message

In [20]:
math_reasoning.refusal

In [21]:
math_reasoning.parsed

MathReasoning(steps=[Step(explanation="The equation given is 8x + 7 = -23. We want to isolate the variable x. To do this, start by removing the constant term on the left side of the equation, which is 7. We'll do this by subtracting 7 from both sides of the equation.", output='8x + 7 - 7 = -23 - 7'), Step(explanation='Subtracting 7 from both sides simplifies the equation. The left side becomes 8x, and the right side becomes -23 - 7, which is -30.', output='8x = -30'), Step(explanation='Now, we need to solve for x by isolating it. Since x is multiplied by 8, we do the opposite operation, which is dividing by 8, on both sides of the equation.', output='x = -30 / 8'), Step(explanation='Simplify the right side of the equation by performing the division. -30 divided by 8 simplifies to -3.75.', output='x = -3.75')], final_answer='x = -3.75')

In [22]:
math_reasoning.parsed.final_answer

'x = -3.75'

In [23]:
math_reasoning.parsed.steps

[Step(explanation="The equation given is 8x + 7 = -23. We want to isolate the variable x. To do this, start by removing the constant term on the left side of the equation, which is 7. We'll do this by subtracting 7 from both sides of the equation.", output='8x + 7 - 7 = -23 - 7'),
 Step(explanation='Subtracting 7 from both sides simplifies the equation. The left side becomes 8x, and the right side becomes -23 - 7, which is -30.', output='8x = -30'),
 Step(explanation='Now, we need to solve for x by isolating it. Since x is multiplied by 8, we do the opposite operation, which is dividing by 8, on both sides of the equation.', output='x = -30 / 8'),
 Step(explanation='Simplify the right side of the equation by performing the division. -30 divided by 8 simplifies to -3.75.', output='x = -3.75')]

### Streaming Structured Output

In [24]:
schema_dict = AnalyzerOutput.schema()
schema_dict["additionalProperties"] = False
schema_dict

/tmp/ipykernel_4511/1884170164.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  schema_dict = AnalyzerOutput.schema()


{'properties': {'classification': {'description': "Either 'Religious' or 'Non-Religious'",
   'enum': ['Religious', 'Non-Religious'],
   'title': 'Classification',
   'type': 'string'},
  'key_topics': {'description': "List of key topics/areas related to the user's question (e.g., theology, jesus, humility, virtues)",
   'items': {'type': 'string'},
   'title': 'Key Topics',
   'type': 'array'},
  'context_requirements': {'description': "A clear explanation of the query's context needs",
   'title': 'Context Requirements',
   'type': 'string'},
  'query_complexity': {'description': "'Low', 'Medium', or 'High' complexity",
   'enum': ['Low', 'Medium', 'High'],
   'title': 'Query Complexity',
   'type': 'string'},
  'reasoning': {'description': 'The Chain of Thought that has been done in order to analyze the user query',
   'title': 'Reasoning',
   'type': 'string'}},
 'required': ['classification',
  'key_topics',
  'context_requirements',
  'query_complexity',
  'reasoning'],
 'title':

In [25]:
text_dict = {
    "format": {
        "type": "json_schema",
        "name": "MyAnalyzerOutput",  # arbitrary name
        "schema": schema_dict
    }
}
text_dict

{'format': {'type': 'json_schema',
  'name': 'MyAnalyzerOutput',
  'schema': {'properties': {'classification': {'description': "Either 'Religious' or 'Non-Religious'",
     'enum': ['Religious', 'Non-Religious'],
     'title': 'Classification',
     'type': 'string'},
    'key_topics': {'description': "List of key topics/areas related to the user's question (e.g., theology, jesus, humility, virtues)",
     'items': {'type': 'string'},
     'title': 'Key Topics',
     'type': 'array'},
    'context_requirements': {'description': "A clear explanation of the query's context needs",
     'title': 'Context Requirements',
     'type': 'string'},
    'query_complexity': {'description': "'Low', 'Medium', or 'High' complexity",
     'enum': ['Low', 'Medium', 'High'],
     'title': 'Query Complexity',
     'type': 'string'},
    'reasoning': {'description': 'The Chain of Thought that has been done in order to analyze the user query',
     'title': 'Reasoning',
     'type': 'string'}},
   'requir

In [26]:
stream = client.responses.create(
    model=MODEL_NAME,
    input=chat_template,
    text=text_dict,
    stream=True,
)

for event in stream:
    if event.type == 'response.refusal.delta':
        print(event.delta, end="")
    elif event.type == 'response.output_text.delta':
        print(event.delta, end="")
    elif event.type == 'response.error':
        print(event.error, end="")
    elif event.type == 'response.completed':
        response = event.response.output

{
  "classification": "Non-Religious",
  "key_topics": ["greeting"],
  "context_requirements": "The query is a simple greeting message and does not imply any religious context.",
  "query_complexity": "Low",
  "reasoning": "The user's message 'Helloo!!' is a casual greeting without any religious elements. It falls under non-religious content, and the simplicity of the greeting means that the query is of low complexity."
}

In [27]:
response

[ResponseReasoningItem(id='rs_67eefa4a0c588191a0db7699d16846980f551bba5511bb39', summary=[], type='reasoning', status=None),
 ResponseOutputMessage(id='msg_67eefa4acbec8191bedd9651ad84bf120f551bba5511bb39', content=[ResponseOutputText(annotations=[], text='{\n  "classification": "Non-Religious",\n  "key_topics": ["greeting"],\n  "context_requirements": "The query is a simple greeting message and does not imply any religious context.",\n  "query_complexity": "Low",\n  "reasoning": "The user\'s message \'Helloo!!\' is a casual greeting without any religious elements. It falls under non-religious content, and the simplicity of the greeting means that the query is of low complexity."\n}', type='output_text')], role='assistant', status='completed', type='message')]

In [28]:
response[0]

ResponseReasoningItem(id='rs_67eefa4a0c588191a0db7699d16846980f551bba5511bb39', summary=[], type='reasoning', status=None)

In [29]:
response[1]

ResponseOutputMessage(id='msg_67eefa4acbec8191bedd9651ad84bf120f551bba5511bb39', content=[ResponseOutputText(annotations=[], text='{\n  "classification": "Non-Religious",\n  "key_topics": ["greeting"],\n  "context_requirements": "The query is a simple greeting message and does not imply any religious context.",\n  "query_complexity": "Low",\n  "reasoning": "The user\'s message \'Helloo!!\' is a casual greeting without any religious elements. It falls under non-religious content, and the simplicity of the greeting means that the query is of low complexity."\n}', type='output_text')], role='assistant', status='completed', type='message')

In [30]:
response[1].role

'assistant'

In [31]:
response[1].status

'completed'

In [32]:
response[1].content[0].text

'{\n  "classification": "Non-Religious",\n  "key_topics": ["greeting"],\n  "context_requirements": "The query is a simple greeting message and does not imply any religious context.",\n  "query_complexity": "Low",\n  "reasoning": "The user\'s message \'Helloo!!\' is a casual greeting without any religious elements. It falls under non-religious content, and the simplicity of the greeting means that the query is of low complexity."\n}'

### Function Calling Response

In [33]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

In [34]:
response = client.responses.create(
    model=MODEL_NAME,
    input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

response

Response(id='resp_67eefa4c1c1c81918cf2ded63d178ee201399e5e5dcbafd3', created_at=1743714892.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='o3-mini-2025-01-31', object='response', output=[ResponseReasoningItem(id='rs_67eefa4cffb48191a0ad2433af706f7f01399e5e5dcbafd3', summary=[], type='reasoning', status=None), ResponseFunctionToolCall(arguments='{"location": "Paris, France"}', call_id='call_StVkKwib4bjRUbJtum2aip1G', name='get_weather', type='function_call', id='fc_67eefa4d79208191918dac892115886a01399e5e5dcbafd3', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'City and country e.g. Bogotá, Colombia'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for a given location.')], top_p=1.0, max_output_tokens=None, pre

In [35]:
print(response.output[1].arguments)
print(response.output[1].id)
print(response.output[1].type)
print(response.output[1].name)

{"location": "Paris, France"}
fc_67eefa4d79208191918dac892115886a01399e5e5dcbafd3
function_call
get_weather


## Langchain LLM-Chains

### Langchain Setup

* Import necessary libraries

In [36]:
from pydantic import BaseModel, Field
from typing import List, Literal

from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.tools import tool

* Initialize Structure Output template

In [37]:
class AnalyzerOutput(BaseModel):
    classification: Literal["Religious", "Non-Religious"] = Field(..., description="Either 'Religious' or 'Non-Religious'")
    key_topics: List[str] = Field(..., description="List of key topics/areas related to the user's question (e.g., theology, jesus, humility, virtues)",)
    context_requirements: str = Field(..., description="A clear explanation of the query's context needs")
    query_complexity: Literal["Low", "Medium", "High"] = Field(..., description="'Low', 'Medium', or 'High' complexity")
    reasoning: str = Field(..., description="The Chain of Thought that has been done in order to analyze the user query")

* Initialize configs

In [38]:
MODEL_NAME = 'o3-mini'

* Initialize prompt templates

In [39]:
analyzer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant that classifies user queries as either religious or non-religious and extracts key topics.",
        ),
        (
            "human",
            "Classify the following query and extract key concepts:\n\nQuery: {query}",
        ),
    ]
)

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant named {assistant_name}. Use the available tools to provide accurate information."
        ),
        (
            "human",
            "{user_input}"
        )
    ]
)

### Agent with Simple Response

In [40]:
llm = ChatOpenAI(model=MODEL_NAME)
analyzer_chain = analyzer_prompt | llm

In [41]:
text = analyzer_chain.invoke('Hello')
text

AIMessage(content='Classification: Non-religious\n\nKey Concepts:\n• Greeting\n• Salutation\n• Casual communication', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 46, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_617f206dd9', 'finish_reason': 'stop', 'logprobs': None}, id='run-490abbfa-d40a-4703-9ec2-8adba68e51f3-0', usage_metadata={'input_tokens': 46, 'output_tokens': 95, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [42]:
print(text.content, '\n\n')
print(text.response_metadata, '\n\n')
print(text.id, '\n\n')
print(text.usage_metadata)

Classification: Non-religious

Key Concepts:
• Greeting
• Salutation
• Casual communication 


{'token_usage': {'completion_tokens': 95, 'prompt_tokens': 46, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_617f206dd9', 'finish_reason': 'stop', 'logprobs': None} 


run-490abbfa-d40a-4703-9ec2-8adba68e51f3-0 


{'input_tokens': 46, 'output_tokens': 95, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}}


### Agent with Simple Response & Tools

In [43]:
@tool
def get_weather(location: str) -> str:
    """Fetches the current weather for a specified location."""
    # Placeholder implementation
    return f"The current weather in {location} is sunny with a temperature of 25°C."

In [44]:
llm = ChatOpenAI(model=MODEL_NAME)
llm_with_tools = llm.bind_tools([get_weather])
chain = chat_prompt | llm_with_tools

In [45]:
inputs = {
    "assistant_name": "Athena",
    "user_input": "What's the weather in Thessaloniki?"
}

In [46]:
response = chain.invoke(inputs)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Yjq1KeYue0cxQuwukbArHuA', 'function': {'arguments': '{"location": "Thessaloniki"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 75, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_617f206dd9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e142ac56-65a0-4552-931a-63174a671c93-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Thessaloniki'}, 'id': 'call_5Yjq1KeYue0cxQuwukbArHuA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 75, 'output_tokens': 26, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

In [47]:
response.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Thessaloniki'},
  'id': 'call_5Yjq1KeYue0cxQuwukbArHuA',
  'type': 'tool_call'}]

In [48]:
response.content

''

In [49]:
response.response_metadata['finish_reason']

'tool_calls'

In [50]:
inputs = {
    "assistant_name": "Athena",
    "user_input": "What's your name?"
}

In [51]:
response = chain.invoke(inputs)
response

AIMessage(content='My name is Athena. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 71, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_617f206dd9', 'finish_reason': 'stop', 'logprobs': None}, id='run-31e4471d-30ed-4ca9-b894-ff339e3e3ce7-0', usage_metadata={'input_tokens': 71, 'output_tokens': 88, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [52]:
response.tool_calls

[]

In [53]:
response.content

'My name is Athena. How can I help you today?'

In [54]:
response.response_metadata['finish_reason']

'stop'

### Agent with Streaming Response

In [55]:
llm = ChatOpenAI(model=MODEL_NAME, streaming=True)
analyzer_chain = analyzer_prompt | llm

In [56]:
response = ''
for token in analyzer_chain.stream('Hello!!'):
    response += token.content
    print(token.content, end='', flush=True)

Classification: Non-religious

Key topics: Greeting, Salutation

The query "Hello!!" is a casual greeting and does not contain any religious content.

In [57]:
response

'Classification: Non-religious\n\nKey topics: Greeting, Salutation\n\nThe query "Hello!!" is a casual greeting and does not contain any religious content.'

### Agent with Streaming Response & Tools

In [58]:
@tool
def get_weather(location: str) -> str:
    """Fetches the current weather for a specified location."""
    # Placeholder implementation
    return f"The current weather in {location} is sunny with a temperature of 25°C."

In [59]:
llm = ChatOpenAI(model=MODEL_NAME, streaming=True)
llm_with_tools = llm.bind_tools([get_weather])
chain = chat_prompt | llm_with_tools

In [60]:
inputs = {
    "assistant_name": "Athena",
    "user_input": "What's your name?"
}

In [61]:
def extract_tool_content(chunk, tool_calls):
    if chunk.tool_call_chunks:
        for tool_chunk in chunk.tool_call_chunks:
            index = tool_chunk['index']
            if index not in tool_calls:
                tool_calls[index] = {
                    'name': tool_chunk['name'],  # might be None at first
                    'args': '',
                    'id': tool_chunk['id']
                }
            if tool_chunk['name'] is not None:
                tool_calls[index]['name'] = tool_chunk['name']
            if tool_chunk['args']:
                tool_calls[index]['args'] += tool_chunk['args']

In [62]:
response = ''
tool_calls = {}

for chunk in chain.stream(inputs):
    response += chunk.content
    print(chunk.content, end='', flush=True)
    
    extract_tool_content(chunk, tool_calls)

My name is Athena. How can I help you today?

In [63]:
tool_calls

{}

In [64]:
response

'My name is Athena. How can I help you today?'

In [65]:
inputs = {
    "assistant_name": "Athena",
    "user_input": "What's the weather like in Athens Greece and what your name?"
}

In [66]:
response = ''
tool_calls = {}

for chunk in chain.stream(inputs):
    response += chunk.content
    print(chunk.content, end='', flush=True)
    
    extract_tool_content(chunk, tool_calls)

In [67]:
tool_calls

{0: {'name': 'get_weather',
  'args': '{"location": "Athens, Greece"}',
  'id': 'call_EqTmQy99WODEgdhrQdCKToPB'}}

In [68]:
response

''

### Agent with Structure Response

In [69]:
analyzer_llm = ChatOpenAI(model=MODEL_NAME).with_structured_output(AnalyzerOutput)

In [70]:
analyzer_chain = analyzer_prompt | analyzer_llm

In [71]:
text = analyzer_chain.invoke('Hello')
text

AnalyzerOutput(classification='Non-Religious', key_topics=['Greetings', 'Salutation'], context_requirements="The query 'Hello' is a simple greeting with no additional context provided, and it does not contain religious themes or specific topics. More context would be needed if the query intended to delve into religious or complex topics.", query_complexity='Low', reasoning="The query 'Hello' is a basic greeting and does not include any religious themes, practices, or terminology. It is classified as non-religious and is considered a low complexity query involving common conversational language.")

In [72]:
print(text.classification, '\n\n')
print(text.context_requirements, '\n\n')
print(text.key_topics, '\n\n')
print(text.query_complexity, '\n\n')
print(text.reasoning, '\n\n')

Non-Religious 


The query 'Hello' is a simple greeting with no additional context provided, and it does not contain religious themes or specific topics. More context would be needed if the query intended to delve into religious or complex topics. 


['Greetings', 'Salutation'] 


Low 


The query 'Hello' is a basic greeting and does not include any religious themes, practices, or terminology. It is classified as non-religious and is considered a low complexity query involving common conversational language. 




In [73]:
type(text)

__main__.AnalyzerOutput

In [74]:
analysis_report = (
    f"{text.reasoning}\n\n"
    f"The analysis of the user query shows that the question is {text.classification} "
    f"and with a {text.query_complexity} complexity. Specifically, the analysis said the following: "
    f"{text.context_requirements} Key topics: {', '.join(text.key_topics)}."
)
print(analysis_report)

The query 'Hello' is a basic greeting and does not include any religious themes, practices, or terminology. It is classified as non-religious and is considered a low complexity query involving common conversational language.

The analysis of the user query shows that the question is Non-Religious and with a Low complexity. Specifically, the analysis said the following: The query 'Hello' is a simple greeting with no additional context provided, and it does not contain religious themes or specific topics. More context would be needed if the query intended to delve into religious or complex topics. Key topics: Greetings, Salutation.
